In [0]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../src")))

from dfutility_helper.utils import add_map
from pyspark.sql.functions import col, expr, lit, avg, sum, max,count, min

In [0]:
params = dbutils.widgets.getAll()
catalog = params.pop('catalog')

In [0]:
city_silver_df = spark.read \
    .format("delta") \
        .table(f"{catalog}.`02-silver`.city_bike_enriched")

In [0]:
daily_station_performance_df = city_silver_df.groupBy("trip_start_date", "start_station_name") \
    .agg(
            avg("trip_duration_mins").alias("avg_trip_duration_mins"),
            count("ride_id").alias("total_trips")
        )
daily_station_performance_df = add_map(daily_station_performance_df, **params)    

In [0]:
daily_ride_df = city_silver_df.groupBy("trip_start_date") \
    .agg(
            max("trip_duration_mins").alias("max_trip_duration_mins"),
            min("trip_duration_mins").alias("min_trip_duration_mins"),
            avg("trip_duration_mins").alias("avg_trip_duration_mins"),
            count("ride_id").alias("total_trips")
        )
daily_ride_df = add_map(daily_ride_df, **params)    

In [0]:
daily_station_performance_df.write \
    .format("delta") \
        .saveAsTable(f"{catalog}.`03-gold`.daily_station_performance")


In [0]:
daily_ride_df.write \
    .format("delta") \
        .saveAsTable(f"{catalog}.`03-gold`.daily_ride_summary")